In [0]:
#install required python libraries and load them
#%run ./install_python_package
%pip install bs4
%pip install requests
%pip install wget
%pip install pyspark

Python interpreter will be restarted.
Collecting bs4
 Downloading bs4-0.0.1.tar.gz (1.1 kB)
Collecting beautifulsoup4
 Downloading beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
Collecting soupsieve>1.2; python_version >= "3.0"
 Downloading soupsieve-2.2.1-py3-none-any.whl (33 kB)
Building wheels for collected packages: bs4
 Building wheel for bs4 (setup.py): started
 Building wheel for bs4 (setup.py): finished with status 'done'
 Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=d46934014a792e419a704780e630e046f3af3bc654fcfe35b34e7035a726a67b
 Stored in directory: /root/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Installing collected packages: soupsieve, beautifulsoup4, bs4
Successfully installed beautifulsoup4-4.9.3 bs4-0.0.1 soupsieve-2.2.1
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3e9e8818-c2bf-4d8b-baa0-7f85e80e4a02/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (2.24.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests) (2020.12.5)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests) (3.0.4)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests) (2.10)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests) (1.25.11)
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3e9e8818-c2bf-4d8b-baa0-7f85e80e4a02/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting wget
 Downloading wget-3.2.zip (10 kB)
Building wheels for collected packages: wget
 Building wheel for wget (setup.py): started
 Building wheel for wget (setup.py): finished with status 'done'
 Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=4648b5d4b545f866119013169c72b198509c711baf9b81157158559ba3cb6ee4
 Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Installing collected packages: wget
Successfully installed wget-3.2
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3e9e8818-c2bf-4d8b-baa0-7f85e80e4a02/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting pyspark
 Downloading pyspark-3.1.1.tar.gz (212.3 MB)
Collecting py4j==0.10.9
 Downloading py4j-0.10.9-py2.py3-none-any.whl (198 kB)
Building wheels for collected packages: pyspark
 Building wheel for pyspark (setup.py): started
 Building wheel for pyspark (setup.py): finished with status 'done'
 Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767607 sha256=08a2a2f6528129ef5c9b36eedd5822ae03eb1d6360ca4131c7d3fa89231fc030
 Stored in directory: /root/.cache/pip/wheels/b3/0e/81/264aeed961e43b9f6ba9ec81c8c540d2d7dccc52c6b51cbf22
Successfully built pyspark
Installing collected packages: py4j, pyspark
Successfully installed py4j-0.10.9 pyspark-3.1.1
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3e9e8818-c2bf-4d8b-baa0-7f85e80e4a02/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import numpy as np
import pandas as pd
import requests
import zipfile
from bs4 import BeautifulSoup
import os
import requests
from os.path import basename
import gzip
import shutil
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.types as st
from pyspark.sql.types import StructType,StructField, StringType

### Steps
  1) Create download folders in DRIVER (Common name for all coins) <br/>
  2) Creates coin specific folder structure in MOUNT  <br/>
  3) Gets the list of zip file download urls for coin type  <br/>
  4) Execute download (Downloaded to DRIVER)  <br/>
  5) Extract the zip files into TSV files (in DRIVER)  <br/>
  6) Move downloaded files, Extracts to coin specific folders in Mount  <br/>
  7) Load TSV files into dataframe  <br/>
  8) Save Selected columns from Dataframe into a CSV file  <br/>
  3) Copy CSV file to FILESTORE to download using Web Browser  <br/>

### Common Methods
These methods do not depend on coin specific folder structure, download things to DRIVER <br/>
1) Create download folders in Driver (Common name for all coins) <br/>
3) Gets the list of zip file download urls for coin type <br/>
4) Execute download (Downloaded to Driver) <br/>
5) Extract the zip files into TSV files (in Driver) <br/>

In [0]:
### Collecting all compressed file download links
def get_download_links(rooturl, key):
    zip_file_urls = []
    url_address = rooturl
    source_code = requests.get(url_address)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, "html.parser")
    extension = 'gz'

    for link in soup.findAll('a'):
        sub_url = link.get('href')
        if sub_url.endswith(extension):
            sub_url = url_address + sub_url +    key
            zip_file_urls.append(sub_url)
    
    zip_file_df = pd.DataFrame({"url_link": zip_file_urls})
    print(zip_file_df.shape[0])
    spark_df = sqlContext.createDataFrame(zip_file_df)
    return(spark_df)
  
    

In [0]:
### Get compressed file links for Bitcoin-Cash Block data       
def get_file_url_links_spark(url_file_path, root_url, key):          
  url_schema_load = st.StructType([st.StructField('url_link', st.StringType(), True)])
  try:
    spark_url_df = spark.read.csv(url_file_path + "url_list.csv", schema= url_schema_load)
    print(spark_url_df.count())
    if spark_url_df.count() > 0:
      return (spark_url_df)
  except:
    print("Url file not found")
    
  spark_url_df = get_download_links(root_url, key)
  spark_url_df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(url_file_path + "url_list.csv")
  return spark_url_df


In [0]:
### Dwonload the compressed file and extract the zip file
def downloader_new(row):
    zip_file_name = ""
    log_file = open("/databricks/driver/ZipDownloads/download_failures.txt", "a")
    download_success= True
    try:
        # Download and write to file.
        zip_url = row["url_link"]
        resp = requests.get(zip_url)
        key_idx = basename(resp.url).find("?key")
        zip_file_name = basename(resp.url)[:key_idx]        
        #print(f"the filename is {zip_file_name}")
        zip_file = open("ZipDownloads/"+zip_file_name, 'wb')
        zip_file.write(resp.content)  
        zip_file.close()      
        extract_TSV_File(zip_file_name)
        dl_success.add(1)  
        download_failed = False
        
    except Exception as ex:
      print (ex)
      download_success = False
      dl_fail.add(1)
    finally:
      if download_success:
         log_file.write(zip_file_name+","+"0" "\n")
      else:
        log_file.write(zip_file_name+","+"1" "\n")
        
      log_file.close()
                

In [0]:
def extract_TSV_File(ZipFileName):  
  block_size=65536
  log_file = open("/databricks/driver/ZipDownloads/Extracts/Extract_failures.txt", "a")
  extract_success= True
  ext = os.path.splitext(ZipFileName)
  dest_file_name = ext[0]
  ext= ext[1]
  if (ext == ".gz"): 
    try:
      dest_file_name_full = "ZipDownloads/Extracts/"+ dest_file_name
      with (gzip.open("ZipDownloads/"+ZipFileName, 'rb')) as s_file, \
        open(dest_file_name_full, 'wb') as d_file:
        shutil.copyfileobj(s_file, d_file, block_size)
    except Exception as ex:
      print("Error in extracting")
      extract_success = False
      print(ex)
    finally:
      if extract_success:
         log_file.write(dest_file_name+","+"0" "\n")
      else:
        log_file.write(dest_file_name+","+"1" "\n")
        
      log_file.close()
      

In [0]:
###Extract tsv files from gz compressed files
def extract_load_into_dataframe(file_directory, bitcastdf):  
  block_size=65536
  for file_name in os.listdir(file_directory): #"/databricks/driver/"):
    ext = os.path.splitext(file_name)
    dest_file_name = ext[0]
    ext= ext[1]
  if (ext == ".gz"): 
    try:
      with (gzip.open(file_name, 'rb')) as s_file, \
        open(dest_file_name, 'wb') as d_file:
        shutil.copyfileobj(s_file, d_file, block_size)
        os.remove("/databricks/driver/" + file_name) 
        tsvdf = pd.read_csv(dest_file_name, sep='\t',  lineterminator='\n', names=None)
        bitcastdf = bitcastdf.append(tsvdf)
    except:
      print("Error in extracting")

In [0]:
### Load Extracted TSV Into a DataFrame
def Load_TSV_into_dataframe(dbfs_file_path):
  distFile = sc.textFile(dbfs_file_path)
  return (distFile)    

In [0]:
def create_driver_folders():
  dbutils.fs.mkdirs("file:/databricks/driver/ZipDownloads")
  dbutils.fs.mkdirs("file:/databricks/driver/ZipDownloads/Extracts")

In [0]:
def cleanup_driver_location():
  dbutils.fs.rm("file:/databricks/driver/ZipDownloads/",  True)
  dbutils.fs.rm("file:/databricks/driver/ZipDownloads/Extracts/",  True)

In [0]:
### Create sql table for URL data frame
def Create_Sql_Table(url_spark_df):
  url_spark_df.createOrReplaceTempView("BC_Block_Url")
  q = """select *
  from BC_Block_Url limit 10
  """
  sub_urldf = spark.sql(q)
  sub_urldf.show(2)

In [0]:
### Executes download command on nodes
dl_success = sc.accumulator(0)
dl_fail = sc.accumulator(0)
dl_skip = sc.accumulator(0)

def download_files_in_url_df(url_df):
  %time url_df.foreach(lambda r: downloader_new(r))
  [dl_success.value, dl_skip.value, dl_fail.value]

In [0]:
def create_bc_block_data_folders(Coin_Mount_Loc, TSV_Mount_Loc, Zip_Mount_Loc):
  ### Down load bitcoin cash block level data
  dbutils.fs.mkdirs(Coin_Mount_Loc)
  dbutils.fs.mkdirs(TSV_Mount_Loc)
  dbutils.fs.mkdirs( Zip_Mount_Loc)

In [0]:
### setup download locations in driver
### Create structure at mnt for bitcoin cash
def setup_folder_strucutre(Coin_Mount_Loc, TSV_Mount_Loc, Zip_Mount_Loc):
  cleanup_driver_location()
  create_driver_folders()
  create_bc_block_data_folders(Coin_Mount_Loc, TSV_Mount_Loc, Zip_Mount_Loc)

In [0]:
### Move downloaed compressed and tsv files from driver to DBFS folders
def move_downloads_from_DRIVER_to_MOUNT(TSVFolderPath, ZIPFolderPath):
  dbutils.fs.mv("file:/databricks/driver/ZipDownloads/Extracts", TSVFolderPath, recurse=True)
  dbutils.fs.mv("file:/databricks/driver/ZipDownloads/", ZIPFolderPath, recurse = True)

### (1) BITCOIN-Cash

In [0]:
BC_CASH_MNT_LOC = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/"
BC_CASH_MNT_TSV_lOC = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData"
BC_CASH_MNT_ZIP_lOC = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles"
destlocation_tsv = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/"

In [0]:
def get_bitcoin_cash_blocks_file_url_links():
  url_file_path = "/mnt/BlockChain/Blocks/Bitcoin-cash/"
  key = '?key=202001ZjMvj8R3BF'
  bc_cash_blocks_root = 'https://gz.blockchair.com/bitcoin-cash/blocks/'
  spark_url_df = get_file_url_links_spark (url_file_path, bc_cash_blocks_root, key)  
  return spark_url_df

In [0]:
### Load all TSV files into a dataframe
def Ceate_df_from_BC_Cash_TSV_files():
  #bc_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/*.tsv")
  bc_block_tsv_rdd_Df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/*.tsv")
  bc_block_tsv_rdd_Df.count()

In [0]:
### create a csv fiel with subset of columns
### copy to filestore to download the file using web browser
def Create_Export_csv_with_subset_of_columns():
  bc_block_tsv_rdd_Df[["ID", "time","size","median_time","guessed_miner","difficulty","fee_total", "fee_total_usd", "reward", "reward_usd"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/bc_block_sub_df_exported.csv")
  dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/bc_block_sub_df_exported.csv", "dbfs:/FileStore/bc_block_sub_df_exported.csv", recurse = True)

In [0]:
### Get url links to downloade compressed files
### Download compressed files, Extract the downloaded zip files
### Logs both download and extraction failures to separate log files
def download_bc_cash_zip_files():
  spark_url_df = get_bitcoin_cash_blocks_file_url_links()
  download_files_in_url_df(spark_url_df)
  
### Move downloaed compressed and tsv files from driver to DBFS folders
def move_downloads_bc_block_to_dbfs():
  move_downloads_from_DRIVER_to_MOUNT(BC_CASH_MNT_TSV_lOC, BC_CASH_MNT_ZIP_lOC)

####Uncomment below code to Execute the download, Extraction, Creation and Export of CSV file for BitCoin-CASH

In [0]:

#setup_folder_strucutre(BC_CASH_MNT_LOC, BC_CASH_MNT_TSV_lOC, BC_CASH_MNT_ZIP_lOC)
#download_bc_cash_zip_files()
#move_downloads_from_DRIVER_to_MOUNT(BC_CASH_MNT_TSV_lOC,BC_CASH_MNT_ZIP_lOC)
#bc_block_tsv_rdd_Df = Ceate_df_from_BC_Cash_TSV_files()
#Create_Export_csv_with_subset_of_columns()



In [0]:
## Code to debug
##df_rows = spark_url_df.limit(10).collect()
##downloader_new(df_rows[2])
###fname = df_rows[1]['url_link']
###print(fname)
###[dl_success.value, dl_skip.value, dl_fail.value]
###print(postcode_of_lat_long(df_rows[0]['Lat'],df_rows[0]['Long_']))

In [0]:
extract_df = spark.read.csv("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles/download_failures.txt")
### Create sql table
extract_df.createOrReplaceTempView("extract_log_table")

In [0]:
bc_block_tsv_rdd_Df = Ceate_df_from_BC_Cash_TSV_files()
bc_block_tsv_rdd_Df.columns

Out[3]: ['id',
 'hash',
 'time',
 'median_time',
 'size',
 'version',
 'version_hex',
 'version_bits',
 'merkle_root',
 'nonce',
 'bits',
 'difficulty',
 'chainwork',
 'coinbase_data_hex',
 'transaction_count',
 'input_count',
 'output_count',
 'input_total',
 'input_total_usd',
 'output_total',
 'output_total_usd',
 'fee_total',
 'fee_total_usd',
 'fee_per_kb',
 'fee_per_kb_usd',
 'cdd_total',
 'generation',
 'generation_usd',
 'reward',
 'reward_usd',
 'guessed_miner']

### (2) For Doge Coin

In [0]:
#https://gz.blockchair.com/dogecoin/blocks/

Out[42]: True

In [0]:
DC_MNT_LOC = "dbfs:/mnt/BlockChain/Blocks/Doge-Coin/"
DC_MNT_TSV_lOC = "dbfs:/mnt/BlockChain/Blocks/Doge-Coin/TSVData"
DC_MNT_ZIP_lOC = "dbfs:/mnt/BlockChain/Blocks/Doge-Coin/ZipFIles"
#destlocation_tsv = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/"

In [0]:
#dbutils.fs.mkdirs("dbfs:/mnt/BlockChain/Blocks/Doge-Coin/")
#dbutils.fs.mv(DC_MNT_LOC, "dbfs:/mnt/BlockChain/Blocks/Doge-Coin/", True)

Out[79]: True

In [0]:
def get_dogecoin_blocks_file_url_links():
  url_file_path = "/mnt/BlockChain/Blocks/Doge-Coin/"
  key = '?key=202001ZjMvj8R3BF'
  dogecoin_blocks_root = 'https://gz.blockchair.com/dogecoin/blocks/'
  spark_url_df = get_file_url_links_spark (url_file_path, dogecoin_blocks_root, key)  
  return spark_url_df

In [0]:
### Load all TSV files into a dataframe
def Ceate_df_from_dogecoin_TSV_files():
  dogecoin_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/Doge-Coin/TSVData/*.tsv")
  dogecoin_block_tsv_rdd_Df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/Doge-Coin/TSVData/*.tsv")
  print(dogecoin_block_tsv_rdd_Df.count())
  return dogecoin_block_tsv_rdd_Df

In [0]:
### Get url links to downloade compressed files
### Download compressed files, Extract the downloaded zip files
### Logs both download and extraction failures to separate log files
def download_dogecoin_zip_files():
  spark_url_df = get_dogecoin_blocks_file_url_links()
  download_files_in_url_df(spark_url_df)
  
### Move downloaed compressed and tsv files from driver to DBFS folders
def move_downloads_dogecoin_block_to_dbfs():
  move_downloads_from_DRIVER_to_MOUNT(DC_MNT_TSV_lOC, DC_MNT_ZIP_lOC)

In [0]:
### Load all TSV files into a dataframe
def Ceate_df_from_DogeCoin_TSV_files():
  #dogecoin_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/Doge_Coin/TSVData/*.tsv")
  dogecoin_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/Doge-Coin/TSVData/*.tsv")
  return dogecoin_block_tsv_rdd_df
  

In [0]:
def Create_Export_DogeCoin_Subset_Columns():
  dogecoin_block_tsv_rdd_df[["ID","time","median_time","size","guessed_miner","difficulty","fee_total","fee_total_usd","reward","reward_usd","is_aux"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Blocks/Doge-Coin/dogecoin_block_tsv_rdd_df.csv")
  dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/Doge-Coin/dogecoin_block_tsv_rdd_df.csv", "dbfs:/FileStore/dogecoin_block_tsv_rdd_df.csv", recurse = True)


#### Get DogeCoin Block Data

In [0]:
## Code to debug
#df_rows = spark_url_df.limit(10).collect()
#downloader_new(df_rows[2])
##move_downloads_from_DRIVER_to_MOUNT(DC_MNT_TSV_lOC, DC_MNT_ZIP_lOC)

###fname = df_rows[1]['url_link']
###print(fname)
###[dl_success.value, dl_skip.value, dl_fail.value]
##p#rint(postcode_of_lat_long(df_rows[0]['Lat'],df_rows[0]['Long_']))

In [0]:
dogecoin_block_tsv_rdd_df.show(5)

+---+--------------------+-------------------+-------------------+----+-------+-----------+------------+--------------------+----------+---------+-------------+--------------------+--------------------+-----------------+-----------+------------+-----------+---------------+--------------+----------------+---------+-------------+----------+--------------+---------+--------------+--------------+--------------+----------+-------------+------+
 id| hash| time| median_time|size|version|version_hex|version_bits| merkle_root| nonce| bits| difficulty| chainwork| coinbase_data_hex|transaction_count|input_count|output_count|input_total|input_total_usd| output_total|output_total_usd|fee_total|fee_total_usd|fee_per_kb|fee_per_kb_usd|cdd_total| generation|generation_usd| reward|reward_usd|guessed_miner|is_aux|
+---+--------------------+-------------------+-------------------+----+-------+-----------+------------+--------------------+----------+---------+-------------+--------------------+--------------------+-----------------+-----------+------------+-----------+---------------+--------------+----------------+---------+-------------+----------+--------------+---------+--------------+--------------+--------------+----------+-------------+------+
 1|82bc68038f6034c05...|2013-12-08 03:55:27|2013-12-08 03:55:27| 190| 1| 1| 1|5f7e779f7600f54e5...|1417875456|504365040|2.44140625E-4|00000000000000000...|04afeda3520102062...| 1| 1| 1| 0| 0.0| 6841600000000| 6.8416| 0| 0.0| 0.0| 0.0| 0.0| 6841600000000| 6.8416| 6841600000000| 6.8416| Unknown| 0|
 2|ea5380659e02a68c0...|2013-12-08 03:55:33|2013-12-08 03:55:27| 190| 1| 1| 1|3b14b76d22a3f2859...|3404207872|504365040|2.44140625E-4|00000000000000000...|04b5eda3520101062...| 1| 1| 1| 0| 0.0|72975200000000| 72.9752| 0| 0.0| 0.0| 0.0| 0.0|72975200000000| 72.9752|72975200000000| 72.9752| Unknown| 0|
 3|76f80a8a81e6f6669...|2013-12-08 03:55:40|2013-12-08 03:55:33| 190| 1| 1| 1|1e10c28574e3b9d70...|3785361152|504365040|2.44140625E-4|00000000000000000...|04bceda3520101062...| 1| 1| 1| 0| 0.0| 1118300000000| 1.1183| 0| 0.0| 0.0| 0.0| 0.0| 1118300000000| 1.1183| 1118300000000| 1.1183| Unknown| 0|
 4|df363f95151d8c38b...|2013-12-08 03:55:43|2013-12-08 03:55:33| 190| 1| 1| 1|9f69a09b940fc7645...| 151130624|504365040|2.44140625E-4|00000000000000000...|04bfeda3520102062...| 1| 1| 1| 0| 0.0|32437800000000| 32.4378| 0| 0.0| 0.0| 0.0| 0.0|32437800000000| 32.4378|32437800000000| 32.4378| Unknown| 0|
 5|f21dc70cb44c18026...|2013-12-08 03:55:49|2013-12-08 03:55:40| 190| 1| 1| 1|bd19df265c7db76eb...|2772437248|504365040|2.44140625E-4|00000000000000000...|04c5eda3520101062...| 1| 1| 1| 0| 0.0|26271100000000| 26.2711| 0| 0.0| 0.0| 0.0| 0.0|26271100000000| 26.2711|26271100000000| 26.2711| Unknown| 0|
+---+--------------------+-------------------+-------------------+----+-------+-----------+------------+--------------------+----------+---------+-------------+--------------------+--------------------+-----------------+-----------+------------+-----------+---------------+--------------+----------------+---------+-------------+----------+--------------+---------+--------------+--------------+--------------+----------+-------------+------+
only showing top 5 rows

In [0]:
##setup_folder_strucutre(DC_MNT_LOC, DC_MNT_TSV_lOC, DC_MNT_ZIP_lOC)
#download_dogecoin_zip_files()
#[dl_success.value, dl_skip.value, dl_fail.value]
#move_downloads_from_DRIVER_to_MOUNT(DC_MNT_TSV_lOC, DC_MNT_ZIP_lOC)
#dogecoin_block_tsv_rdd_df = Ceate_df_from_DogeCoin_TSV_files()
#Create_Export_DogeCoin_Subset_Columns()

In [0]:
#len(dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Doge_Coin/TSVData"))
dogecoin_block_tsv_rdd_df.columns

Out[68]: ['id',
 'hash',
 'time',
 'median_time',
 'size',
 'version',
 'version_hex',
 'version_bits',
 'merkle_root',
 'nonce',
 'bits',
 'difficulty',
 'chainwork',
 'coinbase_data_hex',
 'transaction_count',
 'input_count',
 'output_count',
 'input_total',
 'input_total_usd',
 'output_total',
 'output_total_usd',
 'fee_total',
 'fee_total_usd',
 'fee_per_kb',
 'fee_per_kb_usd',
 'cdd_total',
 'generation',
 'generation_usd',
 'reward',
 'reward_usd',
 'guessed_miner',
 'is_aux']

In [0]:
dogecoin_block_tsv_rdd_df.count()

Out[71]: 3662748

In [0]:

##%fs rm -r "/mnt/BlockChain/Blocks/Bitcoin-cash/url_list.csv"
#dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/url_list.csv")
#dbutils.fs.ls("file:/databricks/driver/ZipDownloads/")
#len(dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles/"))
#len(dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData"))
#dbutils.fs.rm("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/bc_block_sub_df_exported.csv", recurse = True)
#bc_block_tsv_rdd_Df[["guessed_miner"]].collect().unique()["guessed_miner"]
#dbutils.fs.ls("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/ZipFIles/download_failures.txt")
#tsv_files_path = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/blockchair_bitcoin-cash_blocks_*.tsv"


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-53423803709552> in <module> 
 9 #tsv_files_path = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/blockchair_bitcoin-cash_blocks_*.tsv" 
 10 
 ---> 11 spark_url_df

 NameError : name 'spark_url_df' is not defined

### (3) Dash coin

https://gz.blockchair.com/dash/blocks/

In [0]:
### 1
D_MNT_LOC = "dbfs:/mnt/BlockChain/Blocks/Dash/"
D_MNT_TSV_lOC = "dbfs:/mnt/BlockChain/Blocks/Dash/TSVData"
D_MNT_ZIP_lOC = "dbfs:/mnt/BlockChain/Blocks/Dash/ZipFIles"
#destlocation_tsv = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/"

In [0]:
### 2
def get_dash_blocks_file_url_links():
  url_file_path = "/mnt/BlockChain/Blocks/Dash/"
  key = '?key=202001ZjMvj8R3BF'
  dogecoin_blocks_root = 'https://gz.blockchair.com/dash/blocks/'
  spark_url_df = get_file_url_links_spark (url_file_path, dogecoin_blocks_root, key)  
  return spark_url_df

In [0]:
### 3
### Load all TSV files into a dataframe
def Ceate_df_from_dash_TSV_files():
  dash_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/dash/TSVData/*.tsv")
  dash_block_tsv_rdd_Df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/dash/TSVData/*.tsv")
  print(dash_block_tsv_rdd_Df.count())
  return dash_block_tsv_rdd_Df

In [0]:
### 4
### Get url links to downloade compressed files
### Download compressed files, Extract the downloaded zip files
### Logs both download and extraction failures to separate log files
def download_dash_zip_files():
  spark_url_df = get_dash_blocks_file_url_links()
  download_files_in_url_df(spark_url_df)
  
### Move downloaed compressed and tsv files from driver to DBFS folders
def move_dash_block_to_dbfs():
  move_downloads_from_DRIVER_to_MOUNT(D_MNT_TSV_lOC, D_MNT_ZIP_lOC)


In [0]:
###5 
### Load all TSV files into a dataframe
def Ceate_df_from_Dash_TSV_files():
  #dogecoin_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/Doge_Coin/TSVData/*.tsv")
  dash_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/Dash/TSVData/*.tsv")
  return dash_block_tsv_rdd_df
  

In [0]:
###6
def Create_Export_Dash_Subset_Columns():
  dash_block_tsv_rdd_df[["ID","time","median_time","size","guessed_miner","difficulty","fee_total","fee_total_usd","reward","reward_usd"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Blocks/Dash/dash_block_tsv_rdd_df.csv")
  dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/Dash/dash_block_tsv_rdd_df.csv", "dbfs:/FileStore/dash_block_tsv_rdd_df.csv", recurse = True)

In [0]:
##setup_folder_strucutre(D_MNT_LOC, D_MNT_TSV_lOC, D_MNT_ZIP_lOC)
##download_dash_zip_files()
##[dl_success.value, dl_skip.value, dl_fail.value]
##move_downloads_from_DRIVER_to_MOUNT(D_MNT_TSV_lOC, D_MNT_ZIP_lOC)
##dash_block_tsv_rdd_df = Ceate_df_from_Dash_TSV_files()
Create_Export_Dash_Subset_Columns()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1986566475133087> in <module> 
 4 ##move_downloads_from_DRIVER_to_MOUNT(D_MNT_TSV_lOC, D_MNT_ZIP_lOC) 
 5 ##dash_block_tsv_rdd_df = Ceate_df_from_Dash_TSV_files() 
 ----> 6 Create_Export_Dash_Subset_Columns ( ) 

 <command-1986566475133086> in Create_Export_Dash_Subset_Columns () 
 1 ###6 
 2 def Create_Export_Dash_Subset_Columns ( ) : 
 ----> 3 dash_block_tsv_rdd_df [ [ "ID" , "time" , "median_time" , "size" , "guessed_miner" , "difficulty" , "fee_total" , "fee_total_usd" , "reward" , "reward_usd" ] ] . coalesce ( 1 ) . write . option ( "header" , "true" ) . csv ( "dbfs:/mnt/BlockChain/Blocks/Dash/dash_block_tsv_rdd_df.csv" ) 
 4 dbutils . fs . cp ( "dbfs:/mnt/BlockChain/Blocks/Dash/dash_block_tsv_rdd_df.csv" , "dbfs:/FileStore/dash_block_tsv_rdd_df.csv" , recurse = True ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in csv (self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep) 
 1369 charToEscapeQuoteEscaping = charToEscapeQuoteEscaping , 
 1370 encoding=encoding, emptyValue=emptyValue, lineSep=lineSep)
 -> 1371 self . _jwrite . csv ( path ) 
 1372 
 1373 def orc ( self , path , mode = None , partitionBy = None , compression = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : path dbfs:/mnt/BlockChain/Blocks/Dash/dash_block_tsv_rdd_df.csv already exists.

In [0]:
[dl_success.value, dl_skip.value, dl_fail.value]

Out[28]: [2628, 0, 0]

In [0]:
dash_block_tsv_rdd_df.count()

Out[30]: 533838

In [0]:
dash_block_tsv_rdd_df.columns

Out[31]: ['id',
 'hash',
 'time',
 'median_time',
 'size',
 'version',
 'version_hex',
 'version_bits',
 'merkle_root',
 'nonce',
 'bits',
 'difficulty',
 'chainwork',
 'coinbase_data_hex',
 'transaction_count',
 'input_count',
 'output_count',
 'input_total',
 'input_total_usd',
 'output_total',
 'output_total_usd',
 'fee_total',
 'fee_total_usd',
 'fee_per_kb',
 'fee_per_kb_usd',
 'cdd_total',
 'generation',
 'generation_usd',
 'reward',
 'reward_usd',
 'guessed_miner',
 'cbtx']

In [0]:
len(dbutils.fs.ls(D_MNT_TSV_lOC))

Out[112]: 2629

(4)  BITCOIN SV

In [0]:
##https://gz.blockchair.com/bitcoin-sv/blocks/

In [0]:
### 2
def get_bitcoin_sv_blocks_file_url_links():
  url_file_path = "/mnt/BlockChain/Blocks/BC-SV/"
  key = '?key=202001ZjMvj8R3BF'
  bcsv_blocks_root = 'https://gz.blockchair.com/bitcoin-sv/blocks/'
  spark_url_df = get_file_url_links_spark (url_file_path, bcsv_blocks_root, key)  
  return spark_url_df

In [0]:
### 3
### Load all TSV files into a dataframe
def Ceate_df_from_BS_SV_TSV_files():
  bc_sv_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/BC-SV/TSVData/*.tsv")
  bc_sv_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/BC-SV/TSVData/*.tsv")
 
  return bc_sv_block_tsv_rdd_df

In [0]:
### 4
### Get url links to downloade compressed files
### Download compressed files, Extract the downloaded zip files
### Logs both download and extraction failures to separate log files
def download_bc_sv_zip_files():
  spark_url_df = get_bitcoin_sv_blocks_file_url_links()
  download_files_in_url_df(spark_url_df)
  

In [0]:
###5 
### Load all TSV files into a dataframe
def Ceate_df_from_bcsv_TSV_files():
  #dogecoin_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/Doge_Coin/TSVData/*.tsv")
  bc_sv_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/BC-SV/TSVData/*.tsv")
  return bc_sv_block_tsv_rdd_df

In [0]:
###6
def Create_Export_BCSV_Subset_Columns():
  df_bcsv_block_tsv_rdd_df[["ID","time","median_time","size","guessed_miner","difficulty","fee_total","fee_total_usd","reward","reward_usd"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Blocks/BC-SV/bcsv_block_tsv_rdd_df.csv")
  dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/BC-SV/bcsv_block_tsv_rdd_df.csv", "dbfs:/FileStore/bcsv_block_tsv_rdd_df.csv", recurse = True)

Execute below code

In [0]:
### 1
BC_SV_MNT_LOC = "dbfs:/mnt/BlockChain/Blocks/BC-SV/"
BC_SV_MNT_TSV_lOC = "dbfs:/mnt/BlockChain/Blocks/BC-SV/TSVData"
BC_SV_MNT_ZIP_lOC = "dbfs:/mnt/BlockChain/Blocks/BC-SV/ZipFIles"
#destlocation_tsv = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/"

In [0]:
##setup_folder_strucutre(BC_SV_MNT_LOC, BC_SV_MNT_TSV_lOC, BC_SV_MNT_ZIP_lOC)
##download_bc_sv_zip_files()
##[dl_success.value, dl_skip.value, dl_fail.value]
##move_downloads_from_DRIVER_to_MOUNT(BC_SV_MNT_TSV_lOC, BC_SV_MNT_ZIP_lOC)
##df_bcsv_block_tsv_rdd_df = Ceate_df_from_bcsv_TSV_files()
##Create_Export_BCSV_Subset_Columns()

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-1026072726888505> in <module> 
 4 ##move_downloads_from_DRIVER_to_MOUNT(BC_SV_MNT_TSV_lOC, BC_SV_MNT_ZIP_lOC) 
 5 ##df_bcsv_block_tsv_rdd_df = Ceate_df_from_bcsv_TSV_files() 
 ----> 6 Create_Export_BCSV_Subset_Columns ( ) 

 <command-1026072726888504> in Create_Export_BCSV_Subset_Columns () 
 2 def Create_Export_BCSV_Subset_Columns ( ) : 
 3 df_bcsv_block_tsv_rdd_df [ [ "ID" , "time" , "median_time" , "size" , "guessed_miner" , "difficulty" , "fee_total" , "fee_total_usd" , "reward" , "reward_usd" ] ] . coalesce ( 1 ) . write . option ( "header" , "true" ) . csv ( "dbfs:/mnt/BlockChain/Blocks/BC-SV/bcsv_block_tsv_rdd_df.csv" ) 
 ----> 4 dbutils . fs . cp ( "dbfs:/mnt/BlockChain/Blocks/BC-SV/df_bcsv_block_tsv_rdd_df.csv" , "dbfs:/FileStore/bcsv_block_tsv_rdd_df.csv" , recurse = True ) 

 /local_disk0/tmp/1617457434419-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 322 exc . __context__ = None 
 323 exc . __cause__ = None 
 --> 324 raise exc
 325 return f_with_exception_handling
 326 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.cp.
: java.io.FileNotFoundException: /mnt/BlockChain/Blocks/BC-SV/df_bcsv_block_tsv_rdd_df.csv
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.getFileStatus(DatabricksFileSystemV1.scala:270)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.getFileStatus(DatabricksFileSystem.scala:201)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cp$2(DBUtilsCore.scala:113)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:80)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cp$1(DBUtilsCore.scala:112)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:80)
	at com.databricks.backend.daemon.dbutils.FSUtils$.cp(DBUtilsCore.scala:112)
	at com.databricks.backend.daemon.dbutils.FSUtils.cp(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
print ([dl_success.value, dl_skip.value, dl_fail.value])

[4468, 0, 0]

In [0]:
df_bcsv_block_tsv_rdd_df.columns

Out[56]: ['id',
 'hash',
 'time',
 'median_time',
 'size',
 'version',
 'version_hex',
 'version_bits',
 'merkle_root',
 'nonce',
 'bits',
 'difficulty',
 'chainwork',
 'coinbase_data_hex',
 'transaction_count',
 'input_count',
 'output_count',
 'input_total',
 'input_total_usd',
 'output_total',
 'output_total_usd',
 'fee_total',
 'fee_total_usd',
 'fee_per_kb',
 'fee_per_kb_usd',
 'cdd_total',
 'generation',
 'generation_usd',
 'reward',
 'reward_usd',
 'guessed_miner']

In [0]:
df_bcsv_block_tsv_rdd_df.count()

Out[62]: 649178

(5) LITE COIN

In [0]:
def get_litecoin_blocks_file_url_links():
  url_file_path = "/mnt/BlockChain/Blocks/LiteCoin/"
  key = '?key=202001ZjMvj8R3BF'
  litecoin_blocks_root = 'https://gz.blockchair.com/litecoin/blocks/'
  spark_url_df = get_file_url_links_spark (url_file_path, litecoin_blocks_root, key)  
  return spark_url_df

In [0]:
### 3
### Load all TSV files into a dataframe
def Ceate_df_from_LiteCoin_TSV_files():
  litecoin_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/LiteCoin/TSVData/*.tsv")
  litecoin_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/LiteCoin/TSVData/*.tsv")
 
  return litecoin_block_tsv_rdd_df

In [0]:
### 4
### Get url links to downloade compressed files
### Download compressed files, Extract the downloaded zip files
### Logs both download and extraction failures to separate log files
def download_litecoin_zip_files():
  spark_url_df = get_litecoin_blocks_file_url_links()
  download_files_in_url_df(spark_url_df)  

In [0]:
###5 
### Load all TSV files into a dataframe
def Ceate_df_from_litecoin_TSV_files():
  litecoin_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/LiteCoin/TSVData/*.tsv")
  litecoin_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/LiteCoin/TSVData/*.tsv")
  return litecoin_block_tsv_rdd_df

In [0]:
###6
def Create_Export_litecoin_Subset_Columns(block_df):
  block_df[["ID","time","median_time","size","guessed_miner","difficulty","fee_total","fee_total_usd","reward","reward_usd"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Blocks/LiteCoin/block_df.csv")
  dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/LiteCoin/block_df.csv", "dbfs:/FileStore/block_df.csv", recurse = True)

Execute below code

In [0]:
### 1
LC_MNT_LOC = "dbfs:/mnt/BlockChain/Blocks/LiteCoin/"
LC_MNT_TSV_lOC = "dbfs:/mnt/BlockChain/Blocks/LiteCoin/TSVData"
LC_MNT_ZIP_lOC = "dbfs:/mnt/BlockChain/Blocks/LiteCoin/ZipFIles"
#destlocation_tsv = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/"

In [0]:
##setup_folder_strucutre(LC_MNT_LOC, LC_MNT_TSV_lOC, LC_MNT_ZIP_lOC)
##download_litecoin_zip_files()
#[dl_success.value, dl_skip.value, dl_fail.value]
##move_downloads_from_DRIVER_to_MOUNT(LC_MNT_TSV_lOC, LC_MNT_ZIP_lOC)
##litecoin_block_tsv_rdd_df = Ceate_df_from_litecoin_TSV_files()
##Create_Export_litecoin_Subset_Columns(litecoin_block_tsv_rdd_df)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-393408036936613> in <module> 
 4 ##move_downloads_from_DRIVER_to_MOUNT(LC_MNT_TSV_lOC, LC_MNT_ZIP_lOC) 
 5 ##litecoin_block_tsv_rdd_df = Ceate_df_from_litecoin_TSV_files() 
 ----> 6 Create_Export_litecoin_Subset_Columns ( litecoin_block_tsv_rdd_df ) 

 <command-393408036936610> in Create_Export_litecoin_Subset_Columns (block_df) 
 1 ###6 
 2 def Create_Export_litecoin_Subset_Columns ( block_df ) : 
 ----> 3 block_df [ [ "ID" , "time" , "median_time" , "size" , "guessed_miner" , "difficulty" , "fee_total" , "fee_total_usd" , "reward" , "reward_usd" ] ] . coalesce ( 1 ) . write . option ( "header" , "true" ) . csv ( "dbfs:/mnt/BlockChain/Blocks/LiteCoin/block_df.csv" ) 
 4 dbutils . fs . cp ( "dbfs:/mnt/BlockChain/Blocks/LiteCoin/block_df.csv" , "dbfs:/FileStore/block_df.csv" , recurse = True ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in csv (self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep) 
 1369 charToEscapeQuoteEscaping = charToEscapeQuoteEscaping , 
 1370 encoding=encoding, emptyValue=emptyValue, lineSep=lineSep)
 -> 1371 self . _jwrite . csv ( path ) 
 1372 
 1373 def orc ( self , path , mode = None , partitionBy = None , compression = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : path dbfs:/mnt/BlockChain/Blocks/LiteCoin/block_df.csv already exists.

In [0]:
litecoin_block_tsv_rdd_df.count()

Out[57]: 572663

In [0]:
litecoin_block_tsv_rdd_df.count()

Out[42]: 572663

In [0]:
[dl_success.value, dl_skip.value, dl_fail.value]

Out[158]: [10005, 0, 10392]

### (6) Ethereum

In [0]:
dl_success.value = 0
dl_skip.value= 0
dl_fail.value = 0

In [0]:
def get_ethereum_blocks_file_url_links():
  url_file_path = "/mnt/BlockChain/Blocks/Ethereum/"
  key = '?key=202001ZjMvj8R3BF'
  litecoin_blocks_root = 'https://gz.blockchair.com/ethereum/blocks/'
  spark_url_df = get_file_url_links_spark (url_file_path, litecoin_blocks_root, key)  
  return spark_url_df

In [0]:
### 3
### Load all TSV files into a dataframe
def Ceate_df_from_ethereum_TSV_files():
  ethereum_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/Ethereum/TSVData/*.tsv")
  ethereum_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/Ethereum/TSVData/*.tsv")
 
  return litecoin_block_tsv_rdd_df

In [0]:
### 4
### Get url links to downloade compressed files
### Download compressed files, Extract the downloaded zip files
### Logs both download and extraction failures to separate log files
def download_ethereum_zip_files():
  spark_url_df = get_ethereum_blocks_file_url_links()
  download_files_in_url_df(spark_url_df)  

In [0]:
###5 
### Load all TSV files into a dataframe
def Ceate_df_from_ethereum_TSV_files():
  ethereum_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/Ethereum/TSVData/*.tsv")
  ethereum_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/Ethereum/TSVData/*.tsv")
  return ethereum_block_tsv_rdd_df

In [0]:
###6
def Create_Export_Ethereum_Subset_Columns():
  df_ethereum_block_tsv_rdd_df[["ID","time","size","miner","difficulty","fee_total","fee_total_usd","reward","reward_usd"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Blocks/Ethereum/df_ethereum_block_tsv_rdd_df.csv")
  dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/Ethereum/df_ethereum_block_tsv_rdd_df.csv", "dbfs:/FileStore/df_ethereum_block_tsv_rdd_df.csv", recurse = True)

Execute below code to get Ethereum

In [0]:
### 1
EC_MNT_LOC = "dbfs:/mnt/BlockChain/Blocks/Ethereum/"
EC_MNT_TSV_lOC = "dbfs:/mnt/BlockChain/Blocks/Ethereum/TSVData"
EC_MNT_ZIP_lOC = "dbfs:/mnt/BlockChain/Blocks/Ethereum/ZipFIles"
#destlocation_tsv = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/"

In [0]:
##setup_folder_strucutre(EC_MNT_LOC, EC_MNT_TSV_lOC, EC_MNT_ZIP_lOC)
##download_ethereum_zip_files()
##[dl_success.value, dl_skip.value, dl_fail.value]
##move_downloads_from_DRIVER_to_MOUNT(EC_MNT_TSV_lOC, EC_MNT_ZIP_lOC)
##df_ethereum_block_tsv_rdd_df = Ceate_df_from_ethereum_TSV_files()
##Create_Export_Ethereum_Subset_Columns()

In [0]:
[dl_success.value, dl_skip.value, dl_fail.value] 

Out[77]: [2075, 0, 0]

In [0]:
display(df_ethereum_block_tsv_rdd_df.show(5))

+---+--------------------+-------------------+----+--------------------+--------------------+-----------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+-----------+-----------------+---------------------------+----------+--------------------+--------------------+---------------+--------------------+------------------------+--------------------+---------------+-------------------+--------------------+---------+-------------+-------------------+----------+
 id| hash| time|size| miner| extra_data_hex| difficulty|gas_used|gas_limit| logs_bloom| mix_hash| nonce| receipts_root| sha3_uncles| state_root|total_difficulty| transactions_root|uncle_count|transaction_count|synthetic_transaction_count|call_count|synthetic_call_count| value_total|value_total_usd|internal_value_total|internal_value_total_usd| generation| generation_usd| uncle_generation|uncle_generation_usd|fee_total|fee_total_usd| reward|reward_usd|
+---+--------------------+-------------------+----+--------------------+--------------------+-----------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+-----------+-----------------+---------------------------+----------+--------------------+--------------------+---------------+--------------------+------------------------+--------------------+---------------+-------------------+--------------------+---------+-------------+-------------------+----------+
 0|d4e56740f876aef8c...|2015-07-30 00:00:00| 540|0x000000000000000...|11bbe8db4e347b4e8...|17179869184| 0| 5000|00000000000000000...|00000000000000000...| 42|56e81f171bcc55a6f...|1dcc4de8dec75d7aa...|d7f8974fb5ac78d9a...| 400000000|56e81f171bcc55a6f...| 0| 0| 8903| 0| 8903|72009995499480000...|7.20099954995E7|72009995499480000...| 7.20099954995E7|72009995499480000...|7.20099954995E7| 0| 0.0| 0| 0.0|5000000000000000000| 5.0|
 1|88e96d4537bea4d9c...|2015-07-30 15:26:28| 537|0x05a56e2d52c8171...|476574682f76312e3...|17171480576| 0| 5000|00000000000000000...|969b900de27b6ac6a...| 6024642674226568900|56e81f171bcc55a6f...|1dcc4de8dec75d7aa...|d67e4d45034304642...| 34351349760|56e81f171bcc55a6f...| 0| 0| 1| 0| 1| 5000000000000000000| 5.0| 5000000000000000000| 5.0| 5000000000000000000| 5.0| 0| 0.0| 0| 0.0|5000000000000000000| 5.0|
 2|b495a1d7e6663152a...|2015-07-30 15:26:57| 544|0xdd2f1e6e498202e...|476574682f76312e3...|17163096064| 0| 5000|00000000000000000...|2f0790c5aa31ab941...|13282234767606459038|56e81f171bcc55a6f...|1dcc4de8dec75d7aa...|4943d941637411107...| 51514445824|56e81f171bcc55a6f...| 0| 0| 1| 0| 1| 5000000000000000000| 5.0| 5000000000000000000| 5.0| 5000000000000000000| 5.0| 0| 0.0| 0| 0.0|5000000000000000000| 5.0|
 3|3d6122660cc824376...|2015-07-30 15:27:28|1079|0x5088d623ba0fcf0...|476574682f76312e3...|17154715646| 0| 5000|00000000000000000...|65e12eec23fe6555e...| 3356101879604872142|56e81f171bcc55a6f...|6b17b938c6e4ef18b...|76ab0b899e8387436...| 68669161470|56e81f171bcc55a6f...| 1| 0| 2| 0| 2| 8906250000000000000| 8.9062| 5156250000000000000| 5.1562| 5156250000000000000| 5.1562|3750000000000000000| 3.75| 0| 0.0|5156250000000000000| 5.1562|
 4|23adf5a3be0f5235b...|2015-07-30 15:27:57|1079|0xc8ebccc5f5689fa...|59617465732052616...|17146339321| 0| 5000|00000000000000000...|06ba40902198357cb...|14002636129949040352|56e81f171bcc55a6f...|83a8da8965660cb6b...|e6d9f6e95a05ee697...| 85815500791|56e81f171bcc55a6f...| 1| 0| 2| 0| 2| 8281250000000000000| 8.2812| 5156250000000000000| 5.1562| 5156250000000000000| 5.1562|3125000000000000000| 3.125| 0| 0.0|5156250000000000000| 5.1562|
+---+--------------------+-------------------+----+--------------------+--------------------+-----------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [0]:
df_ethereum_block_tsv_rdd_df.columns

Out[79]: ['id',
 'hash',
 'time',
 'size',
 'miner',
 'extra_data_hex',
 'difficulty',
 'gas_used',
 'gas_limit',
 'logs_bloom',
 'mix_hash',
 'nonce',
 'receipts_root',
 'sha3_uncles',
 'state_root',
 'total_difficulty',
 'transactions_root',
 'uncle_count',
 'transaction_count',
 'synthetic_transaction_count',
 'call_count',
 'synthetic_call_count',
 'value_total',
 'value_total_usd',
 'internal_value_total',
 'internal_value_total_usd',
 'generation',
 'generation_usd',
 'uncle_generation',
 'uncle_generation_usd',
 'fee_total',
 'fee_total_usd',
 'reward',
 'reward_usd']

In [0]:
dbutils.fs.ls("file:/databricks/driver/")

Out[105]: [FileInfo(path='file:/databricks/driver/conf/', name='conf/', size=4096),
 FileInfo(path='file:/databricks/driver/eventlogs/', name='eventlogs/', size=4096),
 FileInfo(path='file:/databricks/driver/ganglia/', name='ganglia/', size=4096),
 FileInfo(path='file:/databricks/driver/logs/', name='logs/', size=4096)]

In [0]:
len(dbutils.fs.ls(LC_MNT_TSV_lOC))

Out[119]: 0

### (7) BIT COIN
https://gz.blockchair.com/bitcoin/blocks/

In [0]:
dl_success.value = 0
dl_skip.value= 0
dl_fail.value = 0

In [0]:
def get_bitcoin_blocks_file_url_links():
  url_file_path = "/mnt/BlockChain/Blocks/BitCoin/"
  key = '?key=202001ZjMvj8R3BF'
  litecoin_blocks_root = 'https://gz.blockchair.com/bitcoin/blocks/'
  spark_url_df = get_file_url_links_spark (url_file_path, litecoin_blocks_root, key)  
  return spark_url_df

In [0]:
### 3
### Load all TSV files into a dataframe
def Ceate_df_from_bitcoin_TSV_files():
  bitcoin_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/BitCoin/TSVData/*.tsv")
  bitcoin_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/BitCoin/TSVData/*.tsv")
 
  return bitcoin_block_tsv_rdd_df

In [0]:
### 4
### Get url links to downloade compressed files
### Download compressed files, Extract the downloaded zip files
### Logs both download and extraction failures to separate log files
def download_bitcoin_zip_files():
  spark_url_df = get_bitcoin_blocks_file_url_links()
  download_files_in_url_df(spark_url_df)  

In [0]:
###5 
### Load all TSV files into a dataframe
def Ceate_df_from_bitcoin_TSV_files():
  bitcoin_block_tsv_rdd = sc.textFile("dbfs:/mnt/BlockChain/Blocks/BitCoin/TSVData/*.tsv")
  bitcoin_block_tsv_rdd_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', delimiter='\t', inferschema='true').load("/mnt/BlockChain/Blocks/BitCoin/TSVData/*.tsv")
  return bitcoin_block_tsv_rdd_df

In [0]:
###6
def Create_Export_bitcoin_Subset_Columns():
  bitcoin_block_tsv_rdd_df[["ID","time","median_time", "size","guessed_miner","difficulty","fee_total","fee_total_usd","reward","reward_usd"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Blocks/BitCoin/bitcoin_block_tsv_rdd_df.csv")
  dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/BitCoin/bitcoin_block_tsv_rdd_df.csv", "dbfs:/FileStore/bitcoin_block_tsv_rdd_df.csv", recurse = True)

Execute below code for BitCoin data

In [0]:
### 1
BC_MNT_LOC = "dbfs:/mnt/BlockChain/Blocks/BitCoin/"
BC_MNT_TSV_lOC = "dbfs:/mnt/BlockChain/Blocks/BitCoin/TSVData"
BC_MNT_ZIP_lOC = "dbfs:/mnt/BlockChain/Blocks/BitCoin/ZipFIles"
#destlocation_tsv = "dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/TSVData/"

In [0]:
##setup_folder_strucutre(BC_MNT_LOC, BC_MNT_TSV_lOC, BC_MNT_ZIP_lOC)
##download_bitcoin_zip_files()
[dl_success.value, dl_skip.value, dl_fail.value]
##move_downloads_from_DRIVER_to_MOUNT(BC_MNT_TSV_lOC, BC_MNT_ZIP_lOC)
##bitcoin_block_tsv_rdd_df = Ceate_df_from_bitcoin_TSV_files()
##Create_Export_bitcoin_Subset_Columns()

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-4390671823815049> in <module> 
 4 ##move_downloads_from_DRIVER_to_MOUNT(BC_MNT_TSV_lOC, BC_MNT_ZIP_lOC) 
 5 ##bitcoin_block_tsv_rdd_df = Ceate_df_from_bitcoin_TSV_files() 
 ----> 6 Create_Export_bitcoin_Subset_Columns ( ) 

 <command-4390671823815046> in Create_Export_bitcoin_Subset_Columns () 
 2 def Create_Export_bitcoin_Subset_Columns ( ) : 
 3 bitcoin_block_tsv_rdd_df [ [ "ID" , "time" , "median_time" , "size" , "guessed_miner" , "difficulty" , "fee_total" , "fee_total_usd" , "reward" , "reward_usd" ] ] . coalesce ( 1 ) . write . option ( "header" , "true" ) . csv ( "dbfs:/mnt/BlockChain/Blocks/BitCoin/bitcoin_block_tsv_rdd_df.csv" ) 
 ----> 4 dbutils . fs . cp ( "dbfs:/mnt/BlockChain/Blocks/Ethereum/bitcoin_block_tsv_rdd_df.csv" , "dbfs:/FileStore/bitcoin_block_tsv_rdd_df.csv" , recurse = True ) 

 /local_disk0/tmp/1617541310602-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 322 exc . __context__ = None 
 323 exc . __cause__ = None 
 --> 324 raise exc
 325 return f_with_exception_handling
 326 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.cp.
: java.io.FileNotFoundException: /mnt/BlockChain/Blocks/Ethereum/bitcoin_block_tsv_rdd_df.csv
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.getFileStatus(DatabricksFileSystemV1.scala:270)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.getFileStatus(DatabricksFileSystem.scala:201)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cp$2(DBUtilsCore.scala:113)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:80)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$cp$1(DBUtilsCore.scala:112)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:80)
	at com.databricks.backend.daemon.dbutils.FSUtils$.cp(DBUtilsCore.scala:112)
	at com.databricks.backend.daemon.dbutils.FSUtils.cp(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
bitcoin_block_tsv_rdd_df.count()

Out[101]: 668527

In [0]:
bitcoin_block_tsv_rdd_df.columns

Out[102]: ['id',
 'hash',
 'time',
 'median_time',
 'size',
 'stripped_size',
 'weight',
 'version',
 'version_hex',
 'version_bits',
 'merkle_root',
 'nonce',
 'bits',
 'difficulty',
 'chainwork',
 'coinbase_data_hex',
 'transaction_count',
 'witness_count',
 'input_count',
 'output_count',
 'input_total',
 'input_total_usd',
 'output_total',
 'output_total_usd',
 'fee_total',
 'fee_total_usd',
 'fee_per_kb',
 'fee_per_kb_usd',
 'fee_per_kwu',
 'fee_per_kwu_usd',
 'cdd_total',
 'generation',
 'generation_usd',
 'reward',
 'reward_usd',
 'guessed_miner']

In [0]:
bitcoin_block_tsv_rdd_df.count()

Out[108]: 668527

In [0]:
[dl_success.value, dl_skip.value, dl_fail.value]

Out[109]: [4469, 0, 0]